In [23]:
import pandas as pd

### Cleaning

In [24]:
# let's take a look at our dataset
documents = pd.read_json("dataset-mie.json")
documents.head()

,title,year,journal_issn,language,abstract,doi,pmid,citation_count,IOSPressVolume,publication_type,authors,keywords,topics,affiliation_countries,affiliations
0,Using an open source observational tool to mea...,2009,0926-9630,eng,Computerization of general practice is an inte...,None,19745467,4,150,[Journal Article],"[De Lusignan, S, Kumarapeli, P, Debar, S, Kush...","[Attitude to Computers, Computer Systems, Deci...","[EPR systems, consultation, primary care compu...",[united kingdom],"[St George's University of London, London SW17..."
1,"Portable devices, sensors and networks: wirele...",2009,0926-9630,eng,The 21st century healthcare systems aim at inv...,None,19745466,7,150,"[Journal Article, Research Support, Non-U.S. G...","[Pharow, P, Blobel, B, Ruotsalainen, P, Peters...","[Health Services, Humans, Internet, Precision ...","[health services, Portable devices, portable d...",[germany],"[eHealth Competence Center, Regensburg Univers..."
2,Archetype-based knowledge management for seman...,2009,0926-9630,eng,Formal modeling of clinical content that can b...,None,19745465,13,150,[Journal Article],"[Garde, S, Chen, R, Leslie, H, Beale, T, McNic...","[Medical Record Linkage, Medical Records Syste...","[compliance templates, templates, archetype re...",[united kingdom],"[Ocean Informatics, London, UK. sebastian.gard..."
3,Is there a common background to support better...,2009,0926-9630,eng,The workshop is proposed by the EFMI WG Health...,None,19745464,0,150,[Journal Article],"[Stoicu-Tivadar, L, Blobel, B, Kern, J, Masic,...","[Education, Europe, Humans, International Coop...","[healthcare services, better healthcare, healt...",[romania],"[University Politehnica Timisoara, Romania. st..."
4,Digital pathology in Europe: coordinating pati...,2009,0926-9630,eng,The COST Action IC0604 Telepathology Network ...,None,19745463,7,150,[Journal Article],"[Garcia Rojo, M, Punys, V, Slodkowska, J, Schr...","[Biomedical Research, Europe, Humans, Medical ...","[research efforts, Anatomic Pathology, Patholo...",[spain],"[Hospital General de Ciudad Real, 13005 Ciudad..."


In [25]:
# explore dataset for different language types
documents["language"].value_counts()

language
eng    4565
ger      41
Name: count, dtype: int64

In [26]:
# only look at english doucments
documents = documents[documents['language'] == 'eng'][['abstract', 'keywords', 'topics', 'pmid']]
documents = documents.set_index('pmid')
documents.head()

,abstract,keywords,topics
pmid,,,
19745467,Computerization of general practice is an inte...,"[Attitude to Computers, Computer Systems, Deci...","[EPR systems, consultation, primary care compu..."
19745466,The 21st century healthcare systems aim at inv...,"[Health Services, Humans, Internet, Precision ...","[health services, Portable devices, portable d..."
19745465,Formal modeling of clinical content that can b...,"[Medical Record Linkage, Medical Records Syste...","[compliance templates, templates, archetype re..."
19745464,The workshop is proposed by the EFMI WG Health...,"[Education, Europe, Humans, International Coop...","[healthcare services, better healthcare, healt..."
19745463,The COST Action IC0604 Telepathology Network ...,"[Biomedical Research, Europe, Humans, Medical ...","[research efforts, Anatomic Pathology, Patholo..."


In [30]:
words_per_document = documents["abstract"].apply(len)
words_per_document.mean()

881.7986856516977

In [ ]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
import string
import re
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import TweetTokenizer, RegexpTokenizer
import nltk

In [29]:
#tokenize

# Contraction map
c_dict = {
    "ain't": "am not",
    "aren't": "are not",
    "can't": "cannot",
    "can't've": "cannot have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'd've": "he would have",
    "he'll": "he will",
    "he'll've": "he will have",
    "he's": "he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",
    "i'd": "I would",
    "i'd've": "I would have",
    "i'll": "I will",
    "i'll've": "I will have",
    "i'm": "I am",
    "i've": "I have",
    "isn't": "is not",
    "it'd": "it had",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so is",
    "that'd": "that would",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there had",
    "there'd've": "there would have",
    "there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we had",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who'll've": "who will have",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'alls": "you alls",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you had",
    "you'd've": "you would have",
    "you'll": "you you will",
    "you'll've": "you you will have",
    "you're": "you are",
    "you've": "you have"
}

# Compiling the contraction dict
c_re = re.compile('(%s)' % '|'.join(c_dict.keys()))

# List of stop words
add_stop = ['said', 'say', '...', 'like', 'cnn', 'ad']
stop_words = ENGLISH_STOP_WORDS.union(add_stop)

# List of punctuation
punc = list(set(string.punctuation))


# Splits words on white spaces (leaves contractions intact) and splits out
# trailing punctuation
def casual_tokenizer(text):
    tokenizer = TweetTokenizer()
    tokens = tokenizer.tokenize(text)
    return tokens


def expandContractions(text, c_re=c_re):
    def replace(match):
        return c_dict[match.group(0)]
    return c_re.sub(replace, text)


def process_text(text):
    text = casual_tokenizer(text)
    text = [each.lower() for each in text]
    text = [re.sub('[0-9]+', '', each) for each in text]
    text = [expandContractions(each, c_re=c_re) for each in text]
    text = [SnowballStemmer('english').stem(each) for each in text]
    text = [w for w in text if w not in punc]
    text = [w for w in text if w not in stop_words]
    text = [each for each in text if len(each) > 1]
    text = [each for each in text if ' ' not in each]
    return text


def top_words(topic, n_top_words):
    return topic.argsort()[:-n_top_words - 1:-1]  


def topic_table(model, feature_names, n_top_words):
    topics = {}
    for topic_idx, topic in enumerate(model.components_):
        t = (topic_idx)
        topics[t] = [feature_names[i] for i in top_words(topic, n_top_words)]
    return pd.DataFrame(topics)


def whitespace_tokenizer(text): 
    pattern = r"(?u)\b\w\w+\b" 
    tokenizer_regex = RegexpTokenizer(pattern)
    tokens = tokenizer_regex.tokenize(text)
    return tokens


# Funtion to remove duplicate words
def unique_words(text): 
    ulist = []
    [ulist.append(x) for x in text if x not in ulist]
    return ulist


def word_count(text):
    return len(str(text).split(' '))

In [ ]:
documents['word_count'] = documents['body'].apply(word_count)
documents['word_count'].mean()

In [ ]:
### tokenize the abstracts


### Run NMF